In [15]:
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 20)
from scipy import ndimage

import imageio
import numpy as np
from scipy import ndimage

Populating the interactive namespace from numpy and matplotlib


In [32]:
class LightFieldImage:
    def __init__(self, SuperImagePath, W, H):
        self.superImage = imageio.imread(SuperImagePath)
        self.lightFieldDimension = (W, H)
    
    def LightFieldImages(self):
        lightFieldM, lightFieldN = self.lightFieldDimension
        superImageH, superImageW, _ = self.superImage.shape
        lightFieldH = superImageH / lightFieldN
        lightFieldW = superImageW / lightFieldM
        
        Images = np.zeros((lightFieldN, lightFieldM, lightFieldH, lightFieldW, 3), np.uint8)        
        
        for y in range(lightFieldN):
            for x in range(lightFieldM):
                YStart, YEnd = y * lightFieldH, (y + 1) * lightFieldH
                XStart, XEnd = x * lightFieldW, (x + 1) * lightFieldW
                Images[y, x] = self.superImage[YStart:YEnd, XStart:XEnd]
        return Images
                
    def CreateApertureView(self):
        superImageH, superImageW, _ = self.superImage.shape
        lightFieldM, lightFieldN = self.lightFieldDimension
        superImageH, superImageW, _ = self.superImage.shape
        lightFieldH = superImageH / lightFieldN
        lightFieldW = superImageW / lightFieldM
        
        View = np.zeros_like(self.superImage)
        
        SImXs = np.tile(np.arange(0, superImageW - lightFieldW + 1, lightFieldW), lightFieldM)
        SImYs = np.repeat(np.arange(0, superImageH - lightFieldH + 1, lightFieldH), lightFieldN)
        
        for Y in range(lightFieldH):
            for X in range(lightFieldW):
                VYStart, VYEnd = (Y * lightFieldN), ((Y + 1) * lightFieldN)
                VXStart, VXEnd = (X * lightFieldM), ((X + 1) * lightFieldM)
                
                DataSlice = self.superImage[SImYs + Y, SImXs + X]
                View[VYStart:VYEnd,VXStart:VXEnd, :] = DataSlice.reshape(lightFieldN, lightFieldM, 3)                
        
        self.apertureView = View
        return View
        
    def PlotApertureView(self):
        imshow(self.apertureView)
        
    def SaveApertureView(self, fName):
        imageio.imsave(fName, self.apertureView)
        
    def EpipolarHorizontalSlice(self, H):
        superImageW = self.superImage.shape[1]
        lightFieldM = self.lightFieldDimension[0]
        lightFieldW = superImageW / lightFieldM
        
        SliceStartXs = np.arange(0, superImageW, lightFieldW)
        Epipolar = np.zeros((lightFieldM, lightFieldW, 3), np.uint8)
        
        for I in range(len(SliceStartXs)):
            Epipolar[I] = self.superImage[H, SliceStartXs[I]:SliceStartXs[I] + lightFieldW]
            
        return Epipolar

    def EpipolarVerticalSlice(self, W):
        superImageH = self.superImage.shape[0]
        lightFieldN = self.lightFieldDimension[1]
        lightFieldH = superImageH / lightFieldN
        
        SliceStartYs = np.arange(0, superImageH , lightFieldH)
        Epipolar = np.zeros((lightFieldH, lightFieldN, 3), np.uint8)
        
        for I in range(len(SliceStartYs)):
            Epipolar[:, I] = self.superImage[SliceStartYs[I]:SliceStartYs[I] + lightFieldH, W]
            
        return Epipolar
    
    def FocalStack(self, shift, radius):
        Images = self.LightFieldImages()
        
        lightFieldN, lightFieldM, lightFieldH, lightFieldW, _ = Images.shape
        Images = Images.astype(np.float)
        Images /= 255.0
        
        centerN = lightFieldN / 2
        centerM = lightFieldM / 2
        
        for n in range(lightFieldN):
            for m in range(lightFieldM):
                translateN, translateM = shift * (centerN - n), shift * (centerM - m)
                Images[n, m] = ndimage.interpolation.shift(Images[n, m], (translateN, translateM, 0), order=1)
            
        StackedImage = np.zeros((lightFieldH, lightFieldW, 3))
        
        for i in range(centerN-radius, centerN+radius+1):
            for j in range(centerM-radius, centerM+radius+1):
                StackedImage += Images[i, j]
        
        StackedImage *= 1.0 / float((1 + radius * 2) ** 2)
        
        return StackedImage
        
View = LightFieldImage('./data/9x9/matrioska.jpg', 9, 9)

In [40]:
shifts = np.arange(-2.0, 2.0, 0.1)
Gif = []
for i in shifts:
    Gif.append(View.FocalStack(i, 4))


In [41]:
writer = imageio.get_writer('./matrioska_fstack.mp4', fps=2)

for Frame in Gif:
    writer.append_data(Frame)
writer.close()